In [1]:
import sys
sys.path.append('./utils/')
from tokenization_utils import *
import json
from tqdm.notebook import tqdm
import jsonlines


In [3]:
a = ['Cake', 'Distribution', 'Problem']
b = ['Pen', 'Distribution', 'Problem']
sm = difflib.SequenceMatcher(None,a,b)
c = sm.ratio()
c

0.6666666666666666

### Steps to update XLCoST data

1. Get map data <br>
path = "/home/mingzhu/CodeModel/g4g/test_get_map_data.ipynb"
2. Check execution rate <br>
path = "/home/mingzhu/CodeModel/CodeGen_cwd/Test_compilation_execution.ipynb"
3. Tokenization <br>
path = "/home/mingzhu/CodeModel/CodeGen_cwd/data_preprocessing.ipynb"
4. Check detok execution rate after tokenization
5. Split dict can be reused <br>
path = "/home/mingzhu/CodeModel/g4g/XLCoST_data/map_data/split_dict.json"
6. Update data on derail
7. Generate pairwise data <br>
8. Sanity check of the pairwise data (empty line, mismatch)
path = "/home/mingzhu/CodeModel/g4g/clean_xlcost.ipynb"
8. Ask Aneesh to generate code search data again
9. Generate codedict <br>
10. Generate function level data <br>


## Get map data

In [ ]:
path = "/home/mingzhu/CodeModel/g4g/test_get_map_data.ipynb"

In [ ]:
from get_map_data_utils import *

root_path = "/home/mingzhu/CodeModel/g4g/"
data_path = root_path + "XLCoST_data_new/XLCoST_data_raw/"
file_extensions = {"Java": ".java", "C++": ".cpp", "C": ".c", 
                   "Python": ".py","Javascript": ".js", "PHP":".php", "C#":".cs"}
output_path = root_path + "map_data_xlcost_raw/"
get_map_data(data_path, output_path)

## Stand-alone Detokenizer

In [ ]:
# On Dhruv machine
path = "/home/reddy/ming/CodeModel/code_prepro.ipynb"
# On rail machine
path = "/home/mingzhu/CodeModel/CodeBERT/GraphCodeBERT/translation/code_prepro.ipynb"

In [45]:
from code_prepro.lang_processors import *
from code_prepro.bpe_modes import *

bpe_model = get_bpe()
file_tokenizers = {"Java": java_tokenizer, "C++": cpp_tokenizer, "C": c_tokenizer, 
                   "Python": py_tokenizer,
                   "Javascript": js_tokenizer, "PHP": php_tokenizer, "C#": cs_tokenizer}
file_detokenizers = {"Java": java_detokenizer, "C++": cpp_detokenizer, "C": c_detokenizer, 
                     "Python": py_detokenizer,
                   "Javascript": js_detokenizer, "PHP": php_detokenizer, "C#": cs_detokenizer}

py_code = "def\nfunctionhahaha"
py_tokens = file_tokenizers['Python'](py_code)
py_code_tokenized = " ".join(py_tokens)
print(py_code_tokenized)
py_tokens_bpe = bpe_model.apply_bpe(py_code_tokenized).split()
print(py_tokens_bpe)
py_code_tokenized_after = " ".join(py_tokens_bpe).replace("@@ ", "")
print(py_code_tokenized_after)
py_code_after = file_detokenizers['Python'](py_code_tokenized_after)
print(py_code_after)


def NEW_LINE functionhahaha NEW_LINE
['def', 'NEW_LINE', 'function@@', 'h@@', 'ah@@', 'ah@@', 'a', 'NEW_LINE']
def NEW_LINE functionhahaha NEW_LINE
def
functionhahaha



### Fix Python Misalignment in Mapping files

In [2]:
# fix_mapping_py715("program", "715-Python", "program_formatted")
fix_mapping_py715("mapping", "715-Python-12", "snippet")

#### Check if it works

In [97]:
# json_dict, id_lang_dic, program_lang_dic = read_tok_file()
program_json_dict, program_id_lang_dic = read_program_tok_file()

In [98]:
py_tokens = program_id_lang_dic['Python']['715']['tokens']
py_code_tokenized = " ".join(py_tokens)
with open("test_bug.txt", 'w') as outfile:
    outfile.write(py_code_tokenized)

### Fix Python Misalignment in Pairwise files

In [ ]:
correct_program = get_fixed_py715("program", "715-Python")
correct_snippet = get_fixed_py715("mapping", "715-Python-12")
fix_pair_data_py715(correct_program, check_program=True)
fix_pair_data_py715(correct_snippet, check_program=False)


#### Check the min seq length in each pair file

In [ ]:
len_lang_dic = {}
thres = 20
num = 10
for lang in langs:
    print(lang)
    if lang == "Python":
        continue
    program_dic = program_id_lang_dic[lang]
    len_list = []
    counter = 0
    for k, dic in program_dic.items():
        if len(dic['tokens']) < thres:
            print(dic['program'])
            counter += 1
        if counter > num:
            break
#         len_list.append(len(dic['tokens']))
#     len_list_sorted = sorted(len_list)
#     len_lang_dic[lang] = len_list_sorted

In [ ]:
import matplotlib.pyplot as plt

for lang in langs:
    print(lang)
    len_list = len_lang_dic[lang][:50]
    
    plt.scatter(range(len(len_list)),len_list)
    plt.show()

# DOBF/Transcoder Data pre-processing pipeline

### Pre-processing into map_data

In [ ]:
# CoST data for public release (with new_line):
path = "/home/mingzhu/CodeModel/g4g/CoST_data/with_new_line/CoST_data.zip"

### Tokenization

In [6]:
# From scratch:
tokenize_data("program", "program_formatted", ["problem_desc"], ["desc_"])
tokenize_data("mapping", "snippet", ["comment", "problem_desc"], ["comment_", "desc_"])
# tokenize_functions("code-dict", "functions")
# -----------------------------------------------------
# Use cached files:
# cache_path  = "/home/mingzhu/CodeModel/g4g/XLCoST_data/map_data/"
# use_cache(cache_path, "program", "program_formatted", ["problem_desc"], ["desc_"])
# use_cache(cache_path, "mapping", "snippet", ["comment", "problem_desc"], ["comment_", "desc_"])

100%|██████████| 11028/11028 [06:20<00:00, 28.99it/s]
11028it [06:23, 28.77it/s]
100%|██████████| 11198/11198 [06:53<00:00, 27.09it/s]
11198it [06:59, 26.72it/s]
100%|██████████| 574/574 [00:21<00:00, 26.78it/s]
574it [00:20, 27.40it/s]
100%|██████████| 10813/10813 [06:48<00:00, 26.49it/s]
10813it [06:59, 25.79it/s]
100%|██████████| 9951/9951 [06:52<00:00, 24.13it/s]
9951it [06:47, 24.43it/s]
100%|██████████| 3553/3553 [02:27<00:00, 24.06it/s]
3553it [02:24, 24.58it/s]
100%|██████████| 10735/10735 [08:09<00:00, 21.95it/s]
10735it [07:48, 22.90it/s]


Java 0
Java 11028
C++ 0
C++ 11198
C 0
C 574
Python 191
Python 10813
Javascript 0
Javascript 9951
PHP 0
PHP 3553
C# 0
C# 10735


 71%|███████   | 73329/103896 [49:11<16:40, 30.54it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

28614it [15:51, 30.26it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

85220it [47:16, 30.72it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookAp

100190it [1:33:46, 17.81it/s]


Java 4
Java 103896
C++ 0
C++ 106631
C 0
C 4461
Python 81
Python 92691
Javascript 0
Javascript 81603
PHP 0
PHP 20648
C# 0
C# 100190


In [ ]:
cache_path  = "/home/mingzhu/CodeModel/g4g/XLCoST_data/"

### Train/Valid/Test Split

#### Create new split while taking problem similarity into consideration (Only run it once)

In [6]:
for lang in langs:
    print(len(program_id_lang_dic[lang])/11265)

0.994052374611629
0.9789613848202396
0.9598757212605415
0.9529516200621394
0.8833555259653795
0.3154016866400355
0.05095428317798491


In [2]:
program_json_dict, program_id_lang_dic = read_program_tok_file()
# Only need to run when new split is needed
sim_dict_path = "./cached_files/sim_dict_desc.pkl"
all_desc_dict = get_all_desc_dict(program_id_lang_dic)
desc_list = list(all_desc_dict.keys())
sim_dict = get_sim_dict(sim_dict_path, desc_list)

# TODO!!!
# TODO!!!
# TODO!!!
# TODO!!!
# TODO!!!
# TODO!!! This is where the duplication comes from
sim_desc_dict = get_desc_similarity(sim_dict, desc_list)
new_sim_desc_dict = group_similar_desc(sim_desc_dict)
# Each item in this list is a group of programs that have same or similar descs.
id_group_list = group_similar_program_id(all_desc_dict, new_sim_desc_dict)
# Find the most representative key in each group as the union id to get a new key_list
new_keys_list, key_mapping_dict = get_union_all_key_list(program_id_lang_dic, id_group_list)
union_program_lang_dic = get_union_program_lang_dic(program_id_lang_dic, new_keys_list)

split_param_test = 0.08
split_param_valid = 0.04
# Only need to run once
# split_dict_union = get_split_common_keys(split_param_test, split_param_valid, union_program_lang_dic)
# split_dict = expand_union_split_dict(split_dict_union, key_mapping_dict)
# save_split_dict(split_dict)

100%|██████████| 14675199/14675199 [00:19<00:00, 771009.11it/s]


['C++', 'Java', 'Python', 'C#', 'Javascript', 'PHP', 'C']
['C++', 'Java', 'Python', 'C#', 'Javascript', 'PHP']
['C++', 'Java', 'Python', 'C#', 'Javascript']


In [3]:
sim_desc_dict

{'Check if a number can be represented as sum of two positive perfect cubes': {'Check if a number can be represented as product of two positive perfect cubes',
  'Check if a number can be represented as sum of two consecutive perfect cubes'},
 'Generate an N': {'Construct an N', 'Find an N'},
 'Count Non': {'Count Distinct Non', 'Non'},
 'Sum of all numbers up to N that are co': {'Product of all numbers up to N that are co'},
 'Count of repeating digits in a given Number': {'Count of Prime digits in a Number',
  'Count of unique digits in a given number N'},
 'Coprime divisors of a number': {'Compositorial of a number',
  'Hyperfactorial of a number',
  'Palindromic divisors of a number'},
 'Honaker Prime Number': {'Additive Prime Number', 'Chen Prime Number'},
 'Check if Matrix sum is prime or not': {'Check if a Matrix is Bitonic or not'},
 'Sum of sum': {'Sum of Bitwise',
  'Sum of N',
  'Sum of Semi',
  'Sum of non',
  'Sum of square'},
 'Program to check if N is a Centered Cubic Nu

In [44]:
current_set = set()
for key, i in key_mapping_dict.items():
    seti = set(i)
    if len(seti & current_set) > 0:
        print(seti & current_set)
        current_set = set()
    current_set = current_set | seti

{'10591'}
{'1838', '1837'}
{'6371', '11134', '11161', '5887', '3616', '10130', '5875', '6024', '6023', '6100', '5876', '5877', '5879', '11150', '9603', '5894', '2769', '9640', '6026', '9654', '6101', '3617', '9689', '5899', '5971', '4954', '5890', '5249', '5987', '6118', '5850', '5992', '5930', '6025', '6027', '5895', '6372', '6130', '5294'}
{'12784', '12761', '12763', '12627', '6515'}
{'13776'}
{'7215', '7211', '7209', '7212', '7200', '7226', '7202', '7206', '7230', '7201'}
{'12471', '4815', '4814', '12699'}
{'3472'}
{'3971', '4231', '7105', '4365', '2106', '4366'}
{'3229'}
{'13678'}
{'13933'}


#### Load existing split

In [4]:
# Split information
# ------------------------------------------------
# C++ 496 922 9890
# C++ 0.04386275203395826 0.08153519632118854 0.8746020516448532
# Java 496 922 9728
# Java 0.04450026915485376 0.08272025838865961 0.8727794724564867
# Python 496 922 9618
# Python 0.0449438202247191 0.08354476259514317 0.8715114171801377
# C# 496 922 9493
# C# 0.045458711392173036 0.08450187883787004 0.870039409769957
# Javascript 496 922 8922
# Javascript 0.047969052224371374 0.08916827852998066 0.8628626692456479
# PHP 207 369 3834
# PHP 0.04693877551020408 0.0836734693877551 0.8693877551020408
# C 103 97 559
# C 0.13570487483530963 0.12779973649538867 0.7364953886693018
# ------------------------------------------------
split_dict = load_split_dict()
show_split_info(split_dict)
test_list, val_list = get_eval_list(split_dict)

C++ 496 922 9890
C++ 0.04386275203395826 0.08153519632118854 0.8746020516448532
Java 496 922 9728
Java 0.04450026915485376 0.08272025838865961 0.8727794724564867
Python 496 922 9618
Python 0.0449438202247191 0.08354476259514317 0.8715114171801377
C# 496 922 9493
C# 0.045458711392173036 0.08450187883787004 0.870039409769957
Javascript 496 922 8922
Javascript 0.047969052224371374 0.08916827852998066 0.8628626692456479
PHP 207 369 3834
PHP 0.04693877551020408 0.0836734693877551 0.8693877551020408
C 103 97 559
C 0.13570487483530963 0.12779973649538867 0.7364953886693018


In [5]:
# Check data leak
test_keys = set()
valid_keys = set()
train_keys = set()
all_keys = set()
for lang in langs:
    print(lang, test_keys & valid_keys, valid_keys & train_keys, test_keys & train_keys)
    test_keys = test_keys | set(split_dict[lang]['test'])
    valid_keys = valid_keys | set(split_dict[lang]['valid'])
    train_keys = train_keys | set(split_dict[lang]['train'])
all_keys = test_keys | valid_keys | train_keys
print(len(all_keys), len(train_keys), len(valid_keys), len(test_keys))

C++ set() set() set()
Java set() set() set()
Python set() set() set()
C# set() set() set()
Javascript set() set() set()
PHP set() set() set()
C set() set() set()
11265 9850 496 919


In [6]:
# Fix the data leak issue in split_dict.json
test_dup_keys = (test_keys & train_keys) | (test_keys & valid_keys)
valid_dup_keys = valid_keys & train_keys
new_test = test_keys - test_dup_keys
new_valid = valid_keys - valid_dup_keys
print(new_test & train_keys)
print(new_valid & train_keys)
print(new_test & new_valid)
print(len(new_test), len(new_valid), len(train_keys))
print(len(new_test) + len(new_valid) + len(train_keys))

tags = ['train', 'valid', 'test']
new_split_dict = {}
for lang in langs:
    new_split_dict[lang] = {}
    for tag in tags:
        new_split_dict[lang][tag] = []
    for key in split_dict[lang]['test']:
        if key in new_test:
            new_split_dict[lang]['test'].append(key)
    for key in split_dict[lang]['valid']:
        if key in new_valid:
            new_split_dict[lang]['valid'].append(key)
    new_split_dict[lang]['train'] = split_dict[lang]['train']
    print(lang, [len(new_split_dict[lang][x]) for x in tags])
    print(lang, [len(split_dict[lang][x]) for x in tags])

C++ [9890, 496, 922]
C++ [9890, 509, 943]
Java [9728, 496, 922]
Java [9728, 509, 943]
Python [9618, 496, 922]
Python [9618, 509, 943]
C# [9493, 496, 922]
C# [9493, 509, 943]
Javascript [8922, 496, 922]
Javascript [8922, 509, 943]
PHP [3834, 207, 369]
PHP [3834, 216, 386]
C [559, 103, 97]
C [559, 104, 100]


In [7]:
save_split_dict(new_split_dict)

### Generate translation data

In [12]:
# read tokenized files
json_dict, id_lang_dic, program_lang_dic = read_tok_file()

# Generate snippet pairs
get_n_snippets(1, data_path, test_list, val_list, program_lang_dic, id_lang_dic)
# -----------------------------------------------------

# Generate program pairs
# get_programs(test_list, val_list, id_lang_dic, program_lang_dic)
program_json_dict, program_id_lang_dic = read_program_tok_file()
get_program_new(data_path, test_list, val_list, program_id_lang_dic)
# -----------------------------------------------------

# Generate snippet pairs
# get_n_snippets(3, test_list, val_list, recursive=True)

# -----------------------------------------------------
# Binarize the pairwise files (For transcoder)
binarize(snippet_data, 
         file_extensions.keys(), "code_prepro/bpe/cpp-java-python/vocab")
binarize(program_data, 
         file_extensions.keys(), "code_prepro/bpe/cpp-java-python/vocab")
# -----------------------------------------------------

# Use the val and test file from pair_data_tok_1
# copy_val_test_files(path1, path2)

Java C++
train 89040
val 4419
test 8059
Java C
train 2996
val 353
test 238
Java Python
train 77759
val 3938
test 7259
Java Javascript
train 69341
val 3826
test 7005
Java PHP
train 17853
val 929
test 1672
Java C#
train 87065
val 4437
test 8011
C++ C
train 3386
val 352
test 222
C++ Python
train 80100
val 3913
test 7228
C++ Javascript
train 69507
val 3808
test 6965
C++ PHP
train 17811
val 923
test 1647
C++ C#
train 85662
val 4408
test 7922
C Python
train 2478
val 311
test 203
C Javascript
train 1875
val 309
test 200
C PHP
train 856
val 271
test 183
C C#
train 2958
val 352
test 238
Python Javascript
train 67219
val 3750
test 6861
Python PHP
train 17616
val 923
test 1655
Python C#
train 75843
val 3922
test 7215
Javascript PHP
train 17117
val 921
test 1617
Javascript C#
train 68093
val 3826
test 6961
PHP C#
train 17873
val 928
test 1668
Java C++
train 9450
val 490
test 901
Java C
train 409
val 59
test 49
Java Python
train 8991
val 471
test 882
Java Javascript
train 8470
val 475
test 881
Java

Skipped 54 empty lines!


@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab


Skipped 54 empty lines!


Saving the data to /home/mingzhu/CodeModel/g4g/XLCoST_data_split/pair_data_tok_1/Java-C++/train.java_sa-cpp_sa.java_sa.pth ...
2315794 words (64461 unique) in 89040 sentences.
3 unknown words (1 unique), covering 0.00% of the data.
->@@: 3
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
Saving the data to /home/mingzhu/CodeModel/g4g/XLCoST_data_split/pair_data_tok_1/Java-C++/train.java_sa-cpp_sa.cpp_sa.pth ...
2152019 words (64461 unique) in 89040 sentences.
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ al

...

30725 words (64461 unique) in 158 sentences.
154 unknown words (154 unique), covering 0.50% of the data.
5718: 1
4729: 1
4034: 1
4033: 1
6482: 1
6473: 1
4623: 1
14024: 1
13932: 1
13915: 1
13868: 1
13850: 1
13543: 1
13516: 1
13461: 1
13266: 1
13265: 1
13242: 1
13201: 1
13054: 1
12375: 1
11823: 1
11816: 1
12908: 1
12883: 1
12829: 1
12607: 1
12458: 1
12378: 1
12374: 1
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
Saving the data to /home/mingzhu/CodeModel/g4g/XLCoST_data_split/pair_data_tok_full/PHP-C#/valid.php_sa-csharp_sa.csharp_sa.pth ...
30421 words (64461 unique) in 158 sentences.
154 unknown wor

Skipped 54 empty lines!


12829: 1
12607: 1
12458: 1
12378: 1
12374: 1
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
Saving the data to /home/mingzhu/CodeModel/g4g/XLCoST_data_split/pair_data_tok_full/PHP-C#/test.php_sa-csharp_sa.php_sa.pth ...


Skipped 54 empty lines!


54958 words (64461 unique) in 307 sentences.
305 unknown words (305 unique), covering 0.55% of the data.
5730: 1
5675: 1
4804: 1
4578: 1
4080: 1
4039: 1
3932: 1
3885: 1
9496: 1
9457: 1
9452: 1
9258: 1
8366: 1
7071: 1
6792: 1
4855: 1
4783: 1
4585: 1
14021: 1
14020: 1
13967: 1
13966: 1
13898: 1
13897: 1
13876: 1
13875: 1
13830: 1
13575: 1
13535: 1
13504: 1
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
@@ already in vocab
Saving the data to /home/mingzhu/CodeModel/g4g/XLCoST_data_split/pair_data_tok_full/PHP-C#/test.php_sa-csharp_sa.csharp_sa.pth ...
55036 words (64461 unique) in 307 sentences.
305 unknown words (305 uniq

### Read Map_data

In [4]:
json_dict, id_lang_dic, program_lang_dic = read_tok_file()
program_json_dict, program_id_lang_dic = read_program_tok_file()

In [ ]:
# Python datapoint 715 should be removed. It has \n tokens that are not handled properly.

### Generate text-code data

In [17]:
# comment-code
get_n_snippets_mono(1, data_path, 'comment', test_list, val_list, program_lang_dic, id_lang_dic)
# program-desc
get_program_new_mono(data_path, "desc", test_list, val_list, program_id_lang_dic)
# desc-comment-program
get_program_new_mono_comment(data_path, "desc", test_list, val_list, id_lang_dic, program_id_lang_dic)

# Binarize the pairwise files
# binarize("/home/mingzhu/CodeModel/g4g/pair_data_tok_1_comment/", 
#          file_extensions.keys(), "data/bpe/cpp-java-python/vocab")
# binarize("/home/mingzhu/CodeModel/g4g/pair_data_tok_full_desc/", 
#          file_extensions.keys(), "data/bpe/cpp-java-python/vocab")

Java
train 91089
val 4460
test 8154
C++
train 93847
val 4432
test 8118
C
train 3763
val 350
test 250
Python
train 81207
val 3946
test 7293
Javascript
train 70649
val 3829
test 7033
PHP
train 18027
val 930
test 1682
C#
train 87583
val 4436
test 8013
Java
train 9623
val 494
test 911
C++
train 9797
val 492
test 909
C
train 463
val 60
test 51
Python
train 9263
val 472
test 887
Javascript
train 8590
val 475
test 886
PHP
train 3087
val 158
test 308
C#
train 9345
val 491
test 899
Java
train 9623
val 494
test 911
C++
train 9797
val 492
test 909
C
train 463
val 60
test 51
Python
train 9263
val 472
test 887
Javascript
train 8590
val 475
test 886
PHP
train 3087
val 158
test 308
C#
train 9345
val 491
test 899


In [ ]:
# TODO:
# Add NL support to transcoder (How did they do it?)
# https://github.com/facebookresearch/CodeGen/blob/
# cbbce280314d687a0b4d38295b3868bc572a574c/CodeXGLUE/Code-Text/code-to-text/code/run.py#L50
# They use pre-trained XLM to handle text. Apparently DOBF checkpoints can be directly loaded into xlm
# 既然这样，我就没必要对text-code dataset做binarize吧？可以直接用它们的脚本
# 不过如果想用transcoder做baseline，还是需要NL支持，还是需要Binarize。

def binarize_mono(root, langs, mono_key, voc_path):
    for lang1 in langs:
        print(lang1)
        lang1_lower = lang_lower[lang1]
        path = root + lang1 + '-' + mono_key + '/'
        fns = os.listdir(path)
        for fn in fns:
            if fn.endswith(".pth"):
                os.remove(os.path.join(path, fn))
        for tag in tags:
            fn_prefix = path + tag + "-" + lang1 + "-" + mono_key
            fn1 = fn_prefix + file_extensions[lang1]
            fn2 = fn_prefix + ".txt"
            if tag != 'train':
                fn1 = fn_prefix + "-trans" + file_extensions[lang1]
                fn2 = fn_prefix + "-trans" + ".txt"
            tag_pth = tag
            if tag == 'val':
                tag_pth = 'valid'
            fn_pth_prefix = path + tag_pth + "." + lang1_lower + "_sa-" + lang2_lower + "_sa."
            fn1_pth = fn_pth_prefix + lang1_lower + "_sa.pth"
            fn2_pth = fn_pth_prefix + lang2_lower + "_sa.pth"
            fn_pth_prefix_alt = path + tag_pth + "." + lang2_lower + "_sa-" + lang1_lower + "_sa."
            fn1_pth_alt = fn_pth_prefix_alt + lang1_lower + "_sa.pth"
            fn2_pth_alt = fn_pth_prefix_alt + lang2_lower + "_sa.pth"

            XLM_preprocess(voc_path, fn1, fn1_pth)
            XLM_preprocess(voc_path, fn2, fn2_pth)
            copyfile(fn1_pth, fn1_pth_alt)
            copyfile(fn2_pth, fn2_pth_alt)

### Dataset Stats

In [18]:
# Average number of tokens in each program
_, id_lang_dic_filtered, _ = read_tok_file_filtered()
program_json_dict_filtered, _ = read_program_tok_file_filtered()

snippet_line_num_dict = {lang:[] for lang in langs}
snippet_tokens_num_dict = {lang:[] for lang in langs}
snippet_comment_tokens_num_dict = {lang:[] for lang in langs}
snippet_num_dict = {}
for lang in langs:
    snippet_num_dict[lang] = len(id_lang_dic_filtered[lang])
    for k, snippet in id_lang_dic_filtered[lang].items():
        lines = snippet['snippet'].split('\n')
        snippet_line_num_dict[lang].append(len(lines))
        snippet_tokens_num_dict[lang].append(len(snippet['tokens']))
        snippet_comment_tokens_num_dict[lang].append(len(snippet['comment_tokens']))
print_stat("Average number of lines", snippet_line_num_dict)
print_stat("Average number of tokens", snippet_tokens_num_dict)
print_stat("Average number of comment tokens", snippet_comment_tokens_num_dict)
print_stat_dict("Number of snippets", snippet_num_dict)


Average number of lines
3.41	3.71	2.41	3.82	3.23	4.0	4.05	3.37	

Average number of tokens
21.52	24.1	21.65	23.06	22.52	28.14	25.37	22.83	

Average number of comment tokens
8.25	8.14	7.97	8.23	7.96	8.45	9.67	8.15	

Number of snippets
106630	103896	92609	100190	81603	20648	4461	510037	


In [19]:
program_snippet_num_dict = {lang:[] for lang in langs}
program_line_num_dict = {lang:[] for lang in langs}
program_tokens_num_dict = {lang:[] for lang in langs}
program_desc_tokens_num_dict = {lang:[] for lang in langs}
program_num_dict = {}

for lang in langs:
    program_num_dict[lang] = len(program_json_dict_filtered[lang])
    for program in program_json_dict_filtered[lang]:
        lines = program['program_formatted'].split('\n')
        program_line_num_dict[lang].append(len(lines))
        program_snippet_num_dict[lang].append(len(program['snippet_ids']))
        program_tokens_num_dict[lang].append(len(program['tokens']))
        program_desc_tokens_num_dict[lang].append(len(program['desc_tokens']))
print_stat("Average number of snippets per program", program_snippet_num_dict)
print_stat("Average number of lines per program", program_line_num_dict)
print_stat("Average number of tokens per program",program_tokens_num_dict)
print_stat("Average number of desc tokens per program",program_desc_tokens_num_dict)
print_stat_dict("Number of programs", program_num_dict)


Average number of snippets per program
9.52	9.42	8.51	9.33	8.2	5.81	7.77	8.81	

Average number of lines per program
32.45	34.93	20.54	35.64	26.47	23.23	31.5	29.71	

Average number of tokens per program
204.97	227.09	188.54	215.29	184.63	163.51	197.95	201.96	

Average number of desc tokens per program
10.68	10.67	10.75	10.7	10.87	9.91	8.19	10.66	

Number of programs
11198	11028	10622	10735	9951	3553	574	57661	


# Multilingual Training

### Create all2all dataset

In [ ]:
# Create all2all dataset
path1 = "/home/mingzhu/CodeModel/g4g/pair_data_tok_full/"
path2 = "/home/mingzhu/CodeModel/g4g/pair_data_tok_full_all_2_one/"

langs = file_extensions.keys()
path_all = path2 +  "All/"
if not os.path.exists(path_all):
    os.makedirs(path_all)
for lang2 in langs:
    path_lang2 = path2 + lang2 + "/"
    fns = os.listdir(path_lang2)
    for fn in fns:
        if fn.endswith('.pth') :
            copyfile(path_lang2 + fn, path_all + fn)


### Create all2one dataset

In [ ]:
# Create all2one dataset
path1 = "/home/mingzhu/CodeModel/g4g/pair_data_tok_full/"
path2 = "/home/mingzhu/CodeModel/g4g/pair_data_tok_full_all_2_one/"

langs = file_extensions.keys()

if not os.path.exists(path2):
    os.makedirs(path2)
    
for lang2 in langs:
    path_new = path2 + lang2 + "/"
    if not os.path.exists(path_new):
        os.makedirs(path_new)
    for lang1 in langs:
        if lang2 == lang1:
            continue
        print(lang1, lang2)
        lang1_lower = lang_lower[lang1]
        lang2_lower = lang_lower[lang2]

        path1_root = path1 + lang1 + '-' + lang2 + '/'
        if not os.path.exists(path1_root):
            path1_root = path1 + lang2 + '-' + lang1 + '/'
        lang_pair = lang1_lower + "_sa-" + lang2_lower + "_sa"
        lang_pair_alt = lang2_lower + "_sa-" + lang1_lower + "_sa"
        fns = os.listdir(path1_root)
        for fn in fns:
            if fn.endswith('.pth') and ((lang_pair in fn) or (lang_pair_alt in fn)) :
                copyfile(path1_root + fn, path_new + fn)
                new_fn_list = fn.split(".")
                copyfile(path_new + fn, path_new + ".".join([new_fn_list[0]] + new_fn_list[-2:]))
          